In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

from collections import OrderedDict

import sys
sys.path.append("..")
import utilities, ontology

# Load brain and text data

In [2]:
act_bin = utilities.load_coordinates().astype(float)
print("Document N={}, Structure N={}".format(act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=114


In [3]:
version = 190325
dtm_bin = utilities.load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(dtm_bin.shape[0], dtm_bin.shape[1]))

Document N=18155, Term N=4107


In [4]:
frameworks = ["data-driven", "rdoc", "dsm"]
list_suffixes = ["_lr", "_opsim", "_opsim"]
circuit_suffixes = ["_lr", "", ""]
lists, circuits = {}, {}
for fw, list_suffix, circuit_suffix in zip(frameworks, list_suffixes, circuit_suffixes):
    lists[fw], circuits[fw] = utilities.load_framework(fw, 
                                                       suffix=list_suffix, 
                                                       circuit_suffix=circuit_suffix)

In [5]:
scores = {fw: utilities.score_lists(lists[fw], dtm_bin, label_var="DOMAIN") for fw in frameworks}

In [6]:
words = []
for fw in frameworks:
    words += list(lists[fw]["TOKEN"])
words = sorted(list(set(words)))
structures = list(act_bin.columns)
print("Term N={}, Structure N={}".format(len(words), len(structures)))

Term N=262, Structure N=114


In [7]:
domains = {fw: list(OrderedDict.fromkeys(lists[fw]["DOMAIN"])) for fw in frameworks}

In [8]:
pmids = act_bin.index.intersection(scores["rdoc"].index).intersection(scores["dsm"].index)
len(pmids)

18155

In [9]:
for fw in frameworks:
    scores[fw] = scores[fw].loc[pmids]

In [10]:
dtm_bin = dtm_bin.loc[pmids, words]
act_bin = act_bin.loc[pmids, structures]

# Load frameworks

In [11]:
systems = {}
for fw in frameworks:
    fw_df = pd.DataFrame(0.0, index=words+structures, columns=domains[fw])
    for dom in domains[fw]:
        for word in lists[fw].loc[lists[fw]["DOMAIN"] == dom, "TOKEN"]:
            fw_df.loc[word, dom] = 1.0
        for struct in structures:
            fw_df.loc[struct, dom] = circuits[fw].loc[struct, dom]
    fw_df[fw_df > 0.0] = 1.0
    systems[fw] = fw_df

# Similarity of RDoC and data-driven systems

## Observed values

In [12]:
from scipy.spatial.distance import dice, cdist

In [13]:
def compute_sim_obs(fw):
    sims = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            sims.loc[k,r] = 1.0 - dice(systems["data-driven"][k], systems[fw][r])
    return sims

In [14]:
sims = compute_sim_obs("rdoc")
sims

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0.297872,0.0666667,0,0.265306,0.288889,0
REWARD,0.144578,0.379747,0,0.0689655,0.126582,0
REACTION_TIME,0.403846,0.5,0.0560748,0.0185185,0.16,0.0392157
MANIPULATION,0,0,0.514851,0,0,0.583333
VISION,0.0454545,0,0.241758,0.23913,0.0238095,0.0697674
HEARING,0.0987654,0,0,0.211765,0.181818,0.151899


## Null distribution

In [15]:
def compute_sim_null(fw, n_iter=10000):
    sims_null = np.empty((len(domains["data-driven"]), len(domains[fw]), n_iter))
    for n in range(n_iter):
        null = np.random.choice(words+structures, 
                                size=len(words+structures), replace=False)
        sims_null[:,:,n] = 1.0 - cdist(systems["data-driven"].loc[null].values.T, 
                                       systems[fw].values.T, metric="dice")
        if n % (float(n_iter) / 10.0) == 0:
            print("Iteration {}".format(n))
    return sims_null

In [16]:
sims_null = compute_sim_null("rdoc")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [17]:
from statsmodels.stats.multitest import multipletests

In [18]:
def compute_sim_fdr(fw, sims, sims_null):
    n_iter = sims_null.shape[2]
    pvals = pd.DataFrame(index=domains["data-driven"], columns=domains[fw])
    for i, k in enumerate(domains["data-driven"]):
        for j, r in enumerate(domains[fw]):
            pvals.loc[k,r] = np.sum(sims_null[i,j,:] > sims.loc[k,r]) / float(n_iter)
    fdrs = multipletests(pvals.values.ravel(), method="fdr_bh")[1]
    fdrs = np.reshape(fdrs, pvals.shape)
    fdrs = pd.DataFrame(fdrs, index=domains["data-driven"], columns=domains[fw])
    return fdrs

In [19]:
fdrs = compute_sim_fdr("rdoc", sims, sims_null)
fdrs

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,0.000514286,0.9996,0.9996,0.00225,0,0.9996
REWARD,0.194143,0,0.9996,0.9996,0.31824,0.9996
REACTION_TIME,0,0,0.9996,0.9996,0.44775,0.9996
MANIPULATION,0.9996,0.9996,0,0.9996,0.9996,0
VISION,0.9996,0.9996,0.0044,0.00589091,0.9996,0.9996
HEARING,0.610306,0.9996,0.9996,0.00589091,0.0354,0.1188


In [20]:
def compute_sim_star(fw, fdrs):
    stars = pd.DataFrame("", index=domains["data-driven"], columns=domains[fw])
    for k in domains["data-driven"]:
        for r in domains[fw]:
            fdr = fdrs.loc[k,r]
            if fdr < 0.05:
                stars.loc[k,r] = "*"
            if fdr < 0.01:
                stars.loc[k,r] = "**"
            if fdr < 0.001:
                stars.loc[k,r] = "***"
    return stars

In [21]:
stars = compute_sim_star("rdoc", fdrs)
stars

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,***,,,**,***,
REWARD,,***,,,,
REACTION_TIME,***,***,,,,
MANIPULATION,,,***,,,***
VISION,,,**,**,,
HEARING,,,,**,*,


## Weights for figure

In [22]:
sims[fdrs < 0.01] * 40

,NEGATIVE_VALENCE,POSITIVE_VALENCE,COGNITIVE_SYSTEMS,SOCIAL_PROCESSES,AROUSAL_REGULATION,SENSORIMOTOR_SYSTEMS
MEMORY,11.9149,NaN,NaN,10.6122,11.5556,NaN
REWARD,NaN,15.1899,NaN,NaN,NaN,NaN
REACTION_TIME,16.1538,20,NaN,NaN,NaN,NaN
MANIPULATION,NaN,NaN,20.5941,NaN,NaN,23.3333
VISION,NaN,NaN,9.67033,9.56522,NaN,NaN
HEARING,NaN,NaN,NaN,8.47059,NaN,NaN


# Similarity of DSM and data-driven systems

## Observed values

In [23]:
sims = compute_sim_obs("dsm")
sims

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0,0,0.0784314,0.0701754,0.241379,0,0.346154,0,0
REWARD,0,0,0.05,0,0,0,0,0.0952381,0.097561
REACTION_TIME,0,0,0.0983607,0.0895522,0.117647,0.166667,0.0322581,0.190476,0.0645161
MANIPULATION,0.266667,0.266667,0.0363636,0,0,0,0,0,0.0357143
VISION,0.3,0,0,0,0,0,0,0,0
HEARING,0.246575,0.139535,0,0,0,0.0540541,0,0,0


## Null distribution

In [24]:
sims_null = compute_sim_null("dsm")

Iteration 0
Iteration 1000
Iteration 2000
Iteration 3000
Iteration 4000
Iteration 5000
Iteration 6000
Iteration 7000
Iteration 8000
Iteration 9000


## False discovery rate

In [25]:
fdrs = compute_sim_fdr("dsm", sims, sims_null)
fdrs

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,0.999,0.971325,0.5589,0.876505,0.0027,0.971325,0,0.971325,0.971325
REWARD,0.999,0.971325,0.798565,0.971325,0.971325,0.971325,0.971325,0.39825,0.394691
REACTION_TIME,0.999,0.977506,0.394691,0.798188,0.454431,0.018225,0.971325,0.0108,0.8412
MANIPULATION,0.0045,0,0.971325,0.977506,0.977506,0.971325,0.971325,0.971325,0.971325
VISION,0,0.971325,0.971325,0.971325,0.971325,0.971325,0.971325,0.971325,0.971325
HEARING,0.00324,0.15,0.971325,0.971325,0.971325,0.71712,0.971325,0.971325,0.971325


In [26]:
stars = compute_sim_star("dsm", fdrs)
stars

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,,,,,**,,***,,
REWARD,,,,,,,,,
REACTION_TIME,,,,,,*,,*,
MANIPULATION,**,***,,,,,,,
VISION,***,,,,,,,,
HEARING,**,,,,,,,,


## Weights for figure

In [29]:
sims[fdrs < 0.01] * 40

,DEVELOPMENTAL,PSYCHOTIC,BIPOLAR,DEPRESSIVE,ANXIETY,OBSESSIVE_COMPULSIVE,TRAUMA_STRESSOR,DISRUPTIVE,SUBSTANCE
MEMORY,NaN,NaN,NaN,NaN,9.65517,NaN,13.8462,NaN,NaN
REWARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
REACTION_TIME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MANIPULATION,10.6667,10.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VISION,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HEARING,9.86301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
